In [2]:
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog

class Task:
    def __init__(self, name, description="", attached_file=None):
        self.name = name
        self.description = description
        self.attached_file = attached_file

class Project:
    def __init__(self, name):
        self.name = name
        self.tasks = []

class TrelloApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Trello Clone")

        self.projects = []
        self.lists = []

        tk.Label(self.master, text="Project").grid(row=0, column=0, padx=10, pady=5)
        tk.Label(self.master, text="Task").grid(row=0, column=1, padx=10, pady=5)

        self.project_listbox = tk.Listbox(self.master, selectmode=tk.SINGLE)
        self.project_listbox.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        self.task_listbox = tk.Listbox(self.master, selectmode=tk.SINGLE)
        self.task_listbox.grid(row=1, column=1, padx=10, pady=10, sticky="nsew")

        self.add_project_button = tk.Button(self.master, text="Add Project", command=self.add_project)
        self.add_project_button.grid(row=2, column=0, padx=5, pady=5, sticky="nsew")

        self.delete_project_button = tk.Button(self.master, text="Delete Project", command=self.delete_project)
        self.delete_project_button.grid(row=3, column=0, padx=5, pady=5, sticky="nsew")

        self.add_task_button = tk.Button(self.master, text="Add Task", command=self.add_task)
        self.add_task_button.grid(row=2, column=1, padx=5, pady=5, sticky="nsew")

        self.delete_task_button = tk.Button(self.master, text="Delete Task", command=self.delete_last_task)
        self.delete_task_button.grid(row=3, column=1, padx=5, pady=5, sticky="nsew")

        self.project_listbox.bind('<ButtonRelease-1>', self.show_project_tasks)
        self.task_listbox.bind("<B1-Motion>", self.on_drag)
        self.task_listbox.bind('<Double-Button-1>', self.on_double_click)

        tk.Label(self.master).grid(row=4, column=0, padx=5, pady=5)

        self.add_list_button = tk.Button(self.master, text="Add List", command=self.add_list)
        self.add_list_button.grid(row=2, column=2, padx=5, pady=5, sticky="nsew")

        self.delete_list_button = tk.Button(self.master, text="Delete List", command=self.delete_last_list)
        self.delete_list_button.grid(row=3, column=2, padx=5, pady=5, sticky="nsew")

        self.list_widgets = []

        self.start_x = 0
        self.start_y = 0
        self.dragged_task = None

    def add_project(self):
        project_name = simpledialog.askstring("Input", "Enter project name:")
        if project_name:
            project = Project(project_name)
            self.projects.append(project)
            self.project_listbox.insert(tk.END, project.name)
            self.update_task_listbox()

    def delete_project(self):
        selected_project_index = self.project_listbox.curselection()
        if selected_project_index:
            project_name = self.project_listbox.get(selected_project_index)
            result = messagebox.askquestion("Delete Project", f"Do you want to delete project: {project_name}?")
            if result == "yes":
                self.projects.pop(selected_project_index[0])
                self.project_listbox.delete(selected_project_index)
                self.task_listbox.delete(0, tk.END)

    def add_task(self):
        selected_project_index = self.project_listbox.curselection()
        if selected_project_index:
            project = self.projects[selected_project_index[0]]
            task_name = simpledialog.askstring("Input", "Enter task name:")
            if task_name:
                task = Task(task_name)
                project.tasks.append(task)
                self.update_task_listbox()

    def delete_last_task(self):
        selected_project_index = self.project_listbox.curselection()
        if selected_project_index:
            project = self.projects[selected_project_index[0]]
            if project.tasks:
                project.tasks.pop()
                self.update_task_listbox()

    def show_project_tasks(self, event):
        self.update_task_listbox()

    def update_task_listbox(self):
        selected_project_index = self.project_listbox.curselection()
        if selected_project_index:
            project = self.projects[selected_project_index[0]]
            self.task_listbox.delete(0, tk.END)
            for task in project.tasks:
                self.task_listbox.insert(tk.END, task.name)

    def add_list(self):
        list_name = simpledialog.askstring("Input", "Enter list name:")
        if list_name:
            self.lists.append(list_name)
            listbox = tk.Listbox(self.master, selectmode=tk.SINGLE)
            listbox.grid(row=1, column=len(self.lists) + 1, padx=10, pady=10, sticky="nsew")
            label = tk.Label(self.master, text=list_name)
            label.grid(row=0, column=len(self.lists) + 1, padx=10, pady=5, sticky="nsew")
            self.list_widgets.append((listbox, label))

    def delete_last_list(self):
        if self.list_widgets:
            last_list_widget, last_label_widget = self.list_widgets.pop()
            last_list_widget.destroy()
            last_label_widget.destroy()

        self.update_task_listbox()

    def on_drag(self, event):
        x, y = event.x, event.y
        widget = event.widget

        selected_task_index = widget.nearest(y)
        selected_list_index = 0

        if selected_task_index >= 0 and selected_list_index >= 0:
            task_name = widget.get(selected_task_index)
            listbox, _ = self.list_widgets[selected_list_index]
            task = Task(task_name)
            listbox.insert(tk.END, task.name)
            widget.delete(selected_task_index)
            self.update_task_listbox()

    def on_select(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def on_double_click(self, event):
        self.edit_task()

    def edit_task(self):
        selected_project_index = self.project_listbox.curselection()
        selected_task_index = self.task_listbox.curselection()

        if selected_project_index and selected_task_index:
            project = self.projects[selected_project_index[0]]
            task = project.tasks[selected_task_index[0]]

            edit_window = tk.Toplevel(self.master)
            edit_window.title("Edit Task")

            tk.Label(edit_window, text="Task Name:").grid(row=0, column=0, padx=5, pady=5)
            task_name_entry = tk.Entry(edit_window)
            task_name_entry.grid(row=0, column=1, padx=5, pady=5)
            task_name_entry.insert(0, task.name)

            tk.Label(edit_window, text="Description:").grid(row=1, column=0, padx=5, pady=5)
            description_text = tk.Text(edit_window, height=4, width=30)
            description_text.grid(row=1, column=1, padx=5, pady=5)
            description_text.insert(tk.END, task.description)

            tk.Button(edit_window, text="Attach File", command=lambda: self.attach_file(edit_window, task)).grid(row=2, column=0, columnspan=2, pady=5)

            tk.Button(edit_window, text="Save", command=lambda: self.save_edited_task(edit_window, project, task, task_name_entry, description_text)).grid(row=3, column=0, columnspan=2, pady=10)

    def attach_file(self, edit_window, task):
        file_path = filedialog.askopenfilename(title="Choose a file")
        if file_path:
            task.attached_file = file_path
            messagebox.showinfo("Attachment", "File attached successfully!")

    def save_edited_task(self, edit_window, project, task, task_name_entry, description_text):
        task.name = task_name_entry.get()
        task.description = description_text.get("1.0", tk.END)
        edit_window.destroy()
        self.update_task_listbox()

if __name__ == "__main__":
    root = tk.Tk()
    app = TrelloApp(root)
    root.mainloop()
